anixxx instagram - @o978x

In [ ]:
!pip install -q playwright nest_asyncio
!playwright install chromium

In [ ]:
import asyncio, random, json, math
from pathlib import Path
from datetime import datetime

import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()


URL = input("Github URL: ").strip()
TOTAL_VISITS = int(input("Number of views ...> 1000 .."))
OUTPUT_DIR = "visit_logs"
HEADLESS = True
BATCH_SIZE = 100
DELAY_BETWEEN_BATCHES = 0.5


USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/119.0"
]

VIEWPORTS = [
    (1280, 720),
    (1366, 768),
    (1536, 864),
    (1024, 768)
]

DEFAULT_HEADERS = {
    "From": "ops@example.com",
    "X-Request-Origin": "web-visitor-bot/1.0"
}


async def do_visit(context, url, visit_index):
    page = await context.new_page()
    page.set_default_navigation_timeout(0)

    try:
        start = datetime.utcnow()
        response = await page.goto(url, wait_until="domcontentloaded")
        end = datetime.utcnow()

        result = {
            "visit": visit_index,
            "timestamp": end.isoformat() + "Z",
            "status": response.status if response else None,
            "url": page.url,
            "load_time_s": round((end - start).total_seconds(), 3)
        }

        print(f"[{visit_index}] {result['status']} | {result['load_time_s']}s")

    except Exception as e:
        result = {
            "visit": visit_index,
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "error": str(e)
        }
        print(f"[{visit_index}] ERROR")

    finally:
        await page.close()

    return result


async def run_batch(browser, url, start_idx, batch_size, log_file):
    context = await browser.new_context(
        user_agent=random.choice(USER_AGENTS),
        viewport=dict(zip(["width", "height"], random.choice(VIEWPORTS))),
        extra_http_headers=DEFAULT_HEADERS
    )

    tasks = [
        do_visit(context, url, i)
        for i in range(start_idx, start_idx + batch_size)
    ]

    results = await asyncio.gather(*tasks)

    with log_file.open("a", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r) + "\n")

    await context.close()
    return results


async def run_visits():
    out = Path(OUTPUT_DIR)
    out.mkdir(parents=True, exist_ok=True)

    log_file = out / "visits_log.jsonl"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=HEADLESS)

        total_batches = math.ceil(TOTAL_VISITS / BATCH_SIZE)
        all_results = []

        for b in range(total_batches):
            start = b * BATCH_SIZE + 1
            size = min(BATCH_SIZE, TOTAL_VISITS - b * BATCH_SIZE)

            print(f"\n🚀 Batch {b+1}/{total_batches} ({size} visits)")
            results = await run_batch(browser, URL, start, size, log_file)
            all_results.extend(results)

            await asyncio.sleep(DELAY_BETWEEN_BATCHES)

        summary = {
            "total": len(all_results),
            "generated_at": datetime.utcnow().isoformat() + "Z"
        }

        (out / "summary.json").write_text(json.dumps(summary, indent=2))
        await browser.close()

        print("\n✅ DONE")
        print("📂 Output:", out.resolve())


await run_visits()